In [1]:
%load_ext autoreload
%autoreload 2

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

%matplotlib inline
import numpy as np
import sys
sys.path.append('../')
import gp

/home/d/miniconda2/envs/NP/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
Using gpu device 0: GeForce GTX TITAN (CNMeM is disabled, CuDNN 4007)
/home/d/miniconda2/envs/NP/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [5]:
e_p = []
p = []

In [3]:
import cPickle as pickle
with open('../nets/IPMLB_FULL.p', 'rb') as f:
    cnn = pickle.load(f)
cnn.uuid = 'IPMLB'

KeyboardInterrupt: (KeyboardInterrupt(), <function _constructor_Function at 0x7f59b8e81848>, (<theano.compile.function_module.FunctionMaker object at 0x7f596e55f050>, [<None>, <None>, <CudaNdarray([[[[ 0.83101124 -0.45216268 -0.92241758]
   [ 0.44926044 -0.79137021 -0.85471934]
   [ 0.62316829 -0.22166604 -0.5351423 ]]

  [[ 0.74415201  0.76112461  0.14936428]
   [ 1.26601923  1.12129915  0.54457217]
   [ 0.73289531  0.6029588   0.08996633]]

  [[-0.20223477 -0.05420067 -0.5176177 ]
   [-0.16366521  0.24429148  0.12103456]
   [ 0.43135822  0.11404235  0.20076309]]

  [[ 0.01085452 -0.14073524  0.35528407]
   [ 0.1014398  -0.21538898 -0.13963255]
   [-0.17260572  0.19859107  0.22175764]]]


 [[[-0.42160168 -0.33796558 -0.10554801]
   [-0.55112571 -0.89287311 -0.78223509]
   [-0.5099296  -0.8778429  -1.06378591]]

  [[-0.62700075 -0.55552375 -0.26756749]
   [-0.1141824  -0.10648482 -0.13077492]
   [ 0.03338762  0.01803556  0.15733203]]

  [[-0.28123182  0.08279371  0.06133154]
   [-0.268

In [39]:
e_p = []
p = []
for z in range(250,300):
    
    image, prob, mask, gold, rhoana = gp.Util.read_section('/home/d/data/cylinderNEW',z)
    
    error_patches, patches = gp.Patch.patchify_maxoverlap(image, prob, mask, rhoana, gold, sample_rate=1, clamp_result=False)
    
    e_p.append(error_patches)
    p.append(patches)    

Loading z00000250.png
Loading z00000251.png
Loading z00000252.png
Loading z00000253.png
Loading z00000254.png


KeyboardInterrupt: 

In [34]:
# for z in range(0,1):
e_p = []
p = []
image, prob, gold, rhoana, bb = gp.Legacy.read_dojo_data()

for z in range(0,10):

    error_patches, patches = gp.Patch.patchify_maxoverlap(image[z], prob[z], np.zeros((image[0].shape[0], image[0].shape[1])), rhoana[z], gold[z], sample_rate=1, clamp_result=False)
    
    e_p.append(error_patches)
    p.append(patches)

a


In [29]:
len(e_p[0]), len(p[0])

(40, 348)

In [40]:
# cconvert to FP format with target
import neuroproof
import neuroproof.FocusedProofreading as fp
graph = fp.Graph('/home/d/FP/dojo_test_NEW/graph.json')


In [52]:
g_n = graph.find_close_bodies(5,0,0.)
zn = []
for n in g_n:
    zn.append(n[1])

In [53]:
np.max(zn)

0.71764704585075378

In [87]:
graphs = []
for z in range(10):
    
    g = fp.Graph('/home/d/FP/dojoNEW/graph_'+str(z)+'.json')
    graphs.append(g)

In [88]:
fp_y_test_proba = []
fp_y_test = []
gt_y_test = []
for z in range(10):
    
    cur_e_p = e_p[z]
    cur_p = p[z]
    
    g = graphs[z]
    
    for pa in cur_e_p:
#         print pa['l'], pa['n']
        
        label1 = pa['l']
        label2 = pa['n']

        graph_neighbors = sorted(g.find_close_bodies(label1,0,0.))

        pred = [ne[1] for ne in graph_neighbors if ne[0] == label2]
        
        fp_y_test.append(np.round(pred))
        fp_y_test_proba.append(pred)
        gt_y_test.append(1)
        
    for pa in cur_p:
#         print pa['l'], pa['n']
        
        label1 = pa['l']
        label2 = pa['n']

        graph_neighbors = sorted(g.find_close_bodies(label1,0,0.))

        pred = [ne[1] for ne in graph_neighbors if ne[0] == label2]
        
        fp_y_test.append(np.round(pred))
        fp_y_test_proba.append(pred)
        gt_y_test.append(0)
        
        

    



In [30]:
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc, precision_recall_fscore_support, f1_score, precision_recall_curve, average_precision_score, zero_one_loss


In [90]:
print 'Precision/Recall:'
print classification_report(np.array(gt_y_test), np.array(fp_y_test))

Precision/Recall:
             precision    recall  f1-score   support

          0       0.94      0.69      0.80      3488
          1       0.14      0.51      0.21       332

avg / total       0.87      0.68      0.75      3820



In [91]:
acc_score = accuracy_score(np.array(gt_y_test), np.array(fp_y_test))
print 'Accuracy Score:', acc_score

Accuracy Score: 0.677486910995


3820

In [92]:
gp_y_test_proba = []
gp_y_test = []
gt_y_test = []
for z in range(10):
    
    cur_e_p = e_p[z]
    cur_p = p[z]
    
    for pa in cur_e_p:
#         print pa['l'], pa['n']
        
        label1 = pa['l']
        label2 = pa['n']

        pred = gp.Patch.test_and_unify([pa], cnn)
        
        gp_y_test.append(np.round(pred))
        gp_y_test_proba.append(pred)
        gt_y_test.append(1)
        
    for pa in cur_p:
#         print pa['l'], pa['n']
        
        label1 = pa['l']
        label2 = pa['n']

        pred = gp.Patch.test_and_unify([pa], cnn)        
        
        gp_y_test.append(np.round(pred))
        gp_y_test_proba.append(pred)
        gt_y_test.append(0)

NameError: name 'cnn' is not defined

In [36]:
print classification_report(np.array(gt_y_test), np.array(gp_y_test))

             precision    recall  f1-score   support

          0       1.00      0.92      0.96      3488
          1       0.54      0.95      0.69       332

avg / total       0.96      0.92      0.93      3820



In [38]:
acc_score = accuracy_score(np.array(gt_y_test), np.array(gp_y_test))
print 'Accuracy Score:', acc_score

Accuracy Score: 0.924607329843
